In [1]:
#imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#list of links to team rosters
teams = ['https://www.pro-football-reference.com/teams/buf/2021_roster.htm', 'https://www.pro-football-reference.com/teams/nwe/2021_roster.htm', 'https://www.pro-football-reference.com/teams/mia/2021_roster.htm', 'https://www.pro-football-reference.com/teams/nyj/2021_roster.htm', 'https://www.pro-football-reference.com/teams/cin/2021_roster.htm', 'https://www.pro-football-reference.com/teams/pit/2021_roster.htm', 'https://www.pro-football-reference.com/teams/cle/2021_roster.htm', 'https://www.pro-football-reference.com/teams/rav/2021_roster.htm', 'https://www.pro-football-reference.com/teams/oti/2021_roster.htm', 'https://www.pro-football-reference.com/teams/clt/2021_roster.htm', 'https://www.pro-football-reference.com/teams/htx/2021_roster.htm', 'https://www.pro-football-reference.com/teams/jax/2021_roster.htm', 'https://www.pro-football-reference.com/teams/kan/2021_roster.htm', 'https://www.pro-football-reference.com/teams/rai/2021_roster.htm', 'https://www.pro-football-reference.com/teams/sdg/2021_roster.htm', 'https://www.pro-football-reference.com/teams/den/2021_roster.htm', 'https://www.pro-football-reference.com/teams/dal/2021_roster.htm', 'https://www.pro-football-reference.com/teams/phi/2021_roster.htm', 'https://www.pro-football-reference.com/teams/was/2021_roster.htm', 'https://www.pro-football-reference.com/teams/nyg/2021_roster.htm', 'https://www.pro-football-reference.com/teams/gnb/2021_roster.htm', 'https://www.pro-football-reference.com/teams/min/2021_roster.htm', 'https://www.pro-football-reference.com/teams/chi/2021_roster.htm', 'https://www.pro-football-reference.com/teams/det/2021_roster.htm', 'https://www.pro-football-reference.com/teams/tam/2021_roster.htm', 'https://www.pro-football-reference.com/teams/nor/2021_roster.htm', 'https://www.pro-football-reference.com/teams/atl/2021_roster.htm', 'https://www.pro-football-reference.com/teams/car/2021_roster.htm', 'https://www.pro-football-reference.com/teams/ram/2021_roster.htm', 'https://www.pro-football-reference.com/teams/sfo/2021_roster.htm', 'https://www.pro-football-reference.com/teams/sea/2021_roster.htm', 'https://www.pro-football-reference.com/teams/crd/2021_roster.htm', ]

In [3]:
#blank df
df = pd.DataFrame()

In [4]:
#loop to populate df with all info from team rosters
for team in teams:
    #scrape roster data from team pages
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(team) #using links from teams list
    soup = BeautifulSoup(driver.page_source)
    driver.quit()
    table = soup.find_all('table')[0]  #This is the index of player roster table
    tab_data = [[celldata.text for celldata in rowdata.find_all(["th","td"])]
                            for rowdata in table.find_all("tr")]
    
        
    team_df = pd.DataFrame(tab_data[1:-1], columns = tab_data[0]) #first row is column labels, last row is totals
    #create new column for Year, to link with contract info later based on Player and Year
    team_df.insert(loc=13, column='Year', value=2021)
    #convert appropriate categories to numbers, with few exceptions errors are unwanted rows in data
    team_df[['No.', 'Age', 'G', 'GS', 'Wt', 'AV']] = team_df[['No.', 'Age', 'G', 'GS', 'Wt', 'AV']].apply(pd.to_numeric, errors='coerce')
    
    df = df.append(team_df) #append resulting df of a team's data to df that will store all teams' data

In [5]:
#reset index
df = df.reset_index()
df = df.drop(['index'], axis=1)

#get rid of rows that do not represent any player
df = df[df['Age'].notna()]

#reset index
df = df.reset_index()
df = df.drop(['index'], axis=1)

#fill empty values to prepare columns for conversions to ints
df['No.'] = df['No.'].fillna(0) #some players don't have a number 
df['Wt'].iloc[1535] = 252 #one player was missing weight data

#Convert floats to integers where logical
df[['No.', 'Age', 'G', 'GS', 'Wt', 'AV']] = df[['No.', 'Age', 'G', 'GS', 'Wt', 'AV']].astype('int64')

#empty string instead of zero for players who don't have a number
df['No.'].replace([0], '', inplace=True)

In [47]:
df

,No.,Player,Age,Pos,G,GS,Wt,Ht,College/Univ,BirthDate,Yrs,AV,Drafted (tm/rnd/yr),Year
0,97,Mario Addison,34,DE,17,0,260,6-3,Troy,9/6/1987,10,2,,2021
1,17,Josh Allen,25,QB,17,17,237,6-5,Wyoming,5/21/1996,3,19,Buffalo Bills / 1st / 7th pick / 2018,2021
2,51,Eli Ankou,27,DT,5,0,325,6-3,UCLA,6/8/1994,4,1,,2021
3,96,Boogie Basham,24,DL,8,0,281,6-3,Wake Forest,12/16/1997,Rook,1,Buffalo Bills / 2nd / 61st pick / 2021,2021
4,2,Tyler Bass,24,K,17,0,183,5-10,Georgia Southern,2/14/1997,1,3,Buffalo Bills / 6th / 188th pick / 2020,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2421,38,James Wiggins,24,S,3,0,205,6-0,Cincinnati,4/26/1997,Rook,0,Arizona Cardinals / 7th / 243rd pick / 2021,2021
2422,87,Maxx Williams,27,TE,5,5,252,6-4,Minnesota,4/12/1994,6,2,Baltimore Ravens / 2nd / 55th pick / 2015,2021
2423,20,Marco Wilson,22,CB,14,13,187,5-11,Florida,3/3/1999,Rook,5,Arizona Cardinals / 4th / 136th pick / 2021,2021
2424,16,Ryan Winslow,27,P,1,0,217,6-5,Pittsburgh,4/30/1994,2,0,,2021


In [46]:
#export to csv
df.to_csv(r'C:\Users\FIDataSci.DESKTOP-5QIEL80\profootballref-dataframe-csv', sep='\t', encoding='utf-8')